# Latend Dirichlet Allocation Projection

### Here, we will examine the LDA space I've created with the job descriptions I have. Then, I will take a job description from a female-dominated field, one of the more neutral ones, and then a very heavily male-dominated field to evaluate their distance from one another. I have imported the first part of my notebook, but only did the LDA Projection here after cleaning.

In [81]:
#initial imports
from collections import Counter
from IPython.display import clear_output
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import re
import scipy
import seaborn as sns
import sys
import time
#BeautifulSoup
from bs4 import BeautifulSoup
#gensim
import bz2
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import multiprocessing
import gensim
from gensim import corpora
from gensim import models
import pickle
import pyLDAvis.gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array
from gensim.models import Phrases
#NLTK
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords # Import the stop word list
from nltk import word_tokenize
from nltk.corpus import wordnet as wn
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
#sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV
from xgboost.sklearn import XGBClassifier
from sklearn import linear_model
#SpaCy
import spacy
spacy.load('en')
from spacy.lang.en import English

In [2]:
df = pd.read_csv('~/Desktop/Final Capstone/jobs_db.csv')
pd.set_option('display.max_colwidth', -1)

In [3]:
df.head(10)

id  \
0  15   
1  16   
2  17   
3  18   
4  19   
5  20   
6  21   
7  22   
8  23   
9  24   

                                                                                                                                                                                                 title  \
0  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                                                                     
1  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                                                                     
2  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                                                                     
3  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing Engineer, New York City</h3>                                                                  
4  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Programmer I - Natural Language Processing</h3>                                                                  
5  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Siri - Natural Language Processing Scientist/Engineer</h3>                                                                
6  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">NLP and Machine Learning Graduate</h3>                                                                                    
7  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Data Scientist, Natural Language Processing (NLP), descriptive statistics, statistical modelling, machine learning</h3>   
8  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Software Engineer, Natural Language Processing /NLP</h3>                                                                  
9  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing and Machine Learning Engineer</h3>                                                            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

Turn our df into X, for later use with modelling, train_test_split and getting a y column.

In [4]:
X = df

## EDA and Data Cleaning

Since we web scraped, the text is in pretty raw format.  To get it into useable shape, I plan to get rid of the html code before tokenizing and lemmatizing it. I'll also check for duplicates, and remove any to keep the results as clean as possible.  From here, it will be easier to determine what words can be added as stop words, or, are meaningful to the analysis in some way.

In [5]:
#Let's check for unique job descriptions. 
#If we drop some duplicates, we'll run our cleaning on less data, which is faster.
X.nunique()

id                 7882
title              3464
job_description    4334
job_class          9   
dtype: int64

In [6]:
X.job_class.unique()

array(['Natural Language Processing', 'Text Mining',
       'Pattern Recognition', 'Machine Learning', 'Computer Vision',
       'Text Analytics', 'Apache Spark', 'Speech Recognition',
       'Neural Networks'], dtype=object)

In [7]:
X.groupby(X['job_class'])['job_description'].count().sort_values()

job_class
Pattern Recognition            792
Text Analytics                 795
Neural Networks                827
Computer Vision                829
Natural Language Processing    881
Apache Spark                   919
Machine Learning               934
Speech Recognition             944
Text Mining                    961
Name: job_description, dtype: int64

In [8]:
#It stands to reason that we would want to keep the unique job descriptions, because titles like "Data Scientist"
#are broadly used in the field and not necessarily problematic for the scope of this project.
X = X.drop_duplicates(['job_description'], keep = 'last')

In [9]:
#Let's check to make sure we dropped what we had intended.
X.nunique()

id                 4334
title              3431
job_description    4334
job_class          9   
dtype: int64

In [10]:
X.groupby(X['job_class'])['job_description'].count().sort_values()

job_class
Text Analytics                 171
Text Mining                    171
Pattern Recognition            321
Natural Language Processing    510
Machine Learning               573
Computer Vision                633
Apache Spark                   645
Speech Recognition             648
Neural Networks                662
Name: job_description, dtype: int64

The sum of our "female dominated fields is 1,500 which is 34.6% of our dataset. I'm going to drop rows that are Computer Vision job descriptions.  This should help even out the imbalance, and that particular field was the most "gender-balanced" out of the male-dominated job descriptions we have above.

In [11]:
#Dropping Computer Vision
X = X[X.job_class != 'Computer Vision']

In [12]:
X.reset_index(drop=True, inplace=True)

In [13]:
X.head(10)

id  \
0  17   
1  18   
2  19   
3  20   
4  21   
5  29   
6  37   
7  38   
8  44   
9  45   

                                                                                                                                                      title  \
0  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                          
1  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing Engineer, New York City</h3>                       
2  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Programmer I - Natural Language Processing</h3>                       
3  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Siri - Natural Language Processing Scientist/Engineer</h3>                     
4  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">NLP and Machine Learning Graduate</h3>                                         
5  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research - Natural Language Processing(NLP) Scientist</h3>                  
6  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Analytical Linguist, Google AI</h3>                                            
7  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">IBM Research AI Residency Program</h3>                                         
8  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research Engineer – Natural Language Processing</h3>                        
9  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Software Engineer: Machine Learning - Natural Language Processing (NLP)</h3>   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [14]:
X.groupby(X['job_class'])['job_description'].count().sort_values()

job_class
Text Analytics                 171
Text Mining                    171
Pattern Recognition            321
Natural Language Processing    510
Machine Learning               573
Apache Spark                   645
Speech Recognition             648
Neural Networks                662
Name: job_description, dtype: int64

Although we lost a large portion of the dataset due to duplicates, it's important to prioritize quality over quantity. I evaluated the dataset to ensure that there would not be a class imbalance problem.  Now, our female-dominated fields comprise 40% of the job descriptions, this is pretty good considering we're working with scraped data. So, we'll proceed with cleaning up the dataset, so we can put it into BoW and Tf-Idf and then run some models on it!

In [15]:
#A quick view of the dataset to see what we might need to clean.
X.head(10)

id  \
0  17   
1  18   
2  19   
3  20   
4  21   
5  29   
6  37   
7  38   
8  44   
9  45   

                                                                                                                                                      title  \
0  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Scientist - Natural Language Processing</h3>                          
1  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Natural Language Processing Engineer, New York City</h3>                       
2  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Research Programmer I - Natural Language Processing</h3>                       
3  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Siri - Natural Language Processing Scientist/Engineer</h3>                     
4  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">NLP and Machine Learning Graduate</h3>                                         
5  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research - Natural Language Processing(NLP) Scientist</h3>                  
6  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Analytical Linguist, Google AI</h3>                                            
7  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">IBM Research AI Residency Program</h3>                                         
8  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">AI Research Engineer – Natural Language Processing</h3>                        
9  <h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">Software Engineer: Machine Learning - Natural Language Processing (NLP)</h3>   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

Lots of HTML script here, as expected. Let's get some of that cleaned up before we tokenize.

In [16]:
X['job_description'] = X['job_description'].replace(r'div class="jobsearch-JobComponent-description icl-u-xs-mt--md"><div></div><div><div><div><b>', '', regex=True).replace(r'</div', '', regex=True)

In [17]:
X['title'] = X['title'].replace(r'<h3 class="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title">', '', regex=True).replace(r'</h3>', '', regex=True)

In [18]:
X.head(10)

id  \
0  17   
1  18   
2  19   
3  20   
4  21   
5  29   
6  37   
7  38   
8  44   
9  45   

                                                                     title  \
0  Research Scientist - Natural Language Processing                          
1  Natural Language Processing Engineer, New York City                       
2  Research Programmer I - Natural Language Processing                       
3  Siri - Natural Language Processing Scientist/Engineer                     
4  NLP and Machine Learning Graduate                                         
5  AI Research - Natural Language Processing(NLP) Scientist                  
6  Analytical Linguist, Google AI                                            
7  IBM Research AI Residency Program                                         
8  AI Research Engineer – Natural Language Processing                        
9  Software Engineer: Machine Learning - Natural Language Processing (NLP)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Much better, our job titles look really nice at this point. Onward!

In [19]:
X['job_description'] = X['job_description'].replace(r'<div class="jobsearch-JobComponent-description icl-u-xs-mt--md">', '', regex=True)


In [20]:
X.sample(10)

id  \
589   1043   
2861  6788   
1204  3094   
2930  6901   
2077  5755   
1000  2655   
587   1040   
3035  7077   
2273  6003   
538   964    

                                                                    title  \
589   Analytics Project Manager                                             
2861  Senior Automation Engineer                                            
1204  Technology Summer 2019 Intern - Machine Learning Developer            
2930  Data Architect                                                        
2077  Senior Security Engineer                                              
1000  Lead Artifical Intelligence/Machine Learning Scientist PhD REQUIRED   
587   Senior Data Scientist                                                 
3035  Maintenance Manager                                                   
2273  Senior Data Engineer                                                  
538   Data Scientist                                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [21]:
len(X)

3701

In [22]:
#This will get rid of most of the html markers that are within <>
def clean_html(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [23]:
start = time.time()
for text in X:
    for i in range(len(X)):
        X['job_description'][i] = clean_html(X['job_description'][i])
end = time.time()
print("Done in", end - start, "seconds")

Done in 4.377253770828247 seconds


In [24]:
X.sample(25)

id  \
2945  6921   
2958  6936   
498   879    
791   2039   
104   244    
2996  6979   
1001  2656   
2489  6292   
3533  7735   
722   1933   
3177  7254   
655   1133   
2943  6918   
2389  6135   
1819  5213   
602   1059   
1387  3365   
909   2194   
3184  7265   
3335  7485   
3117  7181   
2226  5938   
1331  3283   
293   578    
3468  7651   

                                                                                                title  \
2945  Sr. Software Development Engineer-Audio                                                           
2958  Applied Scientist- AWS AI                                                                         
498   Sr Software Engineer - Algorithmic Bidding                                                        
791   Summer Research Program Intern--BS/MS Candidates                                                  
104   Saudi Arabic speaking Associate Linguist J468                                                     
2996  Senior Software Development Engineer                                                              
1001  Software Engineer, Sr Principal                                                                   
2489  CORRECTIONAL COUNSELOR 2*-020619-173778                                                           
3533  Intern - Design Enablement                                                                        
722   Instagram - Manager, Software Engineering                                                         
3177  Senior Data Scientist                                                                             
655   Data Scientist / Senior Data Scientist (Information Technology)                                   
2943  Senior iOS Engineer                                                                               
2389  People Technology - Compliance Technology - Control Room Compliance - Project Manager\Developer   
1819  Director of Recruiting, Engineering                                                               
602   Senior Data Scientist                                                                             
1387  Scientist, Machine Learning                                                                       
909   Senior Financial Accountant                                                                       
3184  Functional Safety Engineer                                                                        
3335  Performance Analytics Manager                                                                     
3117  Machine Learning Research Scientist                                                               
2226  SR DATA SCIENTIST - ADVANCED TECHNOLOGIES                                                         
1331  Machine Learning Engineer                                                                         
293   Systems Manager                                                                                   
3468  SDM, Ad Measurement                                                                               

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

That worked like a charm! Let's continue cleaning.

In [25]:
X['job_description'] = X['job_description'].replace(r'\n', ' ', regex=True)

In [26]:
X.sample(25)

id  \
514   909    
3220  7304   
2550  6367   
1436  3426   
1399  3385   
392   732    
588   1041   
2818  6727   
2321  6061   
1991  5635   
189   378    
2261  5990   
3281  7401   
278   548    
2349  6091   
3354  7512   
1015  2689   
957   2264   
1147  2991   
1339  3295   
3312  7457   
3113  7176   
886   2165   
2536  6349   
134   291    

                                                                     title  \
514   Advanced Analytics Enablement Cognitive - Consultant                   
3220  Data Science Manager                                                   
2550  Sr Business Development Manager, Alexa Voice Service                   
1436  AI Engineer(Machine Learning)                                          
1399  MACHINE LEARNING ENGINEER                                              
392   Data Scientist II                                                      
588   Data Scientist                                                         
2818  Loss Mitigation Specialist I                                           
2321  Senior Data Engineer                                                   
1991  Cognitive Machine Learning Senior Manager                              
189   Assistant or Associate Professor in Data Analytics and Public Policy   
2261  Lead Engineer- Java/Scala                                              
3281  Relative Value Trader                                                  
278   Data Scientist                                                         
2349  Senior Software Development Engineer - Alexa Smart Home                
3354  Intern                                                                 
1015  Machine Learning Intern                                                
957   Overnight Security Officer $13.50 SRO Position Full Time               
1147  Machine Learning Specialist                                            
1339  Data Science Graduate Internship - Summer 2019                         
3312  Analyst Load Research / Forecasting 1                                  
3113  Software Engineer - Systems and Platform                               
886   Assistant, Associate or Full Professor, Learning Sciences              
2536  Bankruptcy Specialist I                                                
134   Leadership Partner - Data &amp; Analytics                              

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [27]:
X['job_description'] = X['job_description'].replace(r'&amp;', ' ', regex=True)

In [28]:
X['title'] = X['title'].replace(r'&amp;', ' ', regex=True)

In [29]:
X.sample(25)

id                                                              title  \
214   438   Assistant Professor-Computer Science/Data Science                   
1107  2899  Engineering Manager, NLP and Applied Machine Learning               
1100  2883  Machine Learning Developer                                          
1624  4486  Voice of the Customer, Digital Text Analyst - Spanish Speaking      
3155  7226  Software Engineer, Computer Vision, Interaction Tracking (Oculus)   
2508  6313  Siri - Program Manager: Personal Productivity                       
3138  7206  Lead Data Scientist                                                 
3145  7214  Software Engineering Co-Op                                          
3559  7763  Computer Systems Engineer                                           
1159  3014  Principal Researcher - Machine Learning                             
890   2170  Perception Engineer                                                 
2532  6345  Conversation Designer Lead - Remote                                 
1299  3233  Curation Editor                                                     
3077  7131  Data Scientist                                                      
1255  3166  Machine Learning Global Program Director                            
1832  5284  Senior Data Engineer, Infrastructure                                
1004  2663  Machine Learning Engineer (Performance) - CoreML                    
1431  3421  Economist                                                           
99    236   Sr. Machine Learning Engineer – Content Knowledge                   
1640  4505  Forensic Analytics - Senior Consultant                              
2674  6550  Shuttle Driver                                                      
1806  5177  Bioinformatics Python Developer                                     
2615  6461  Dock Hand - Natick Boathouse                                        
1858  5462  Data Scientist (Secret Clearance Required)                          
1929  5554  Data Engineer                                                       

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [30]:
X['job_description'] = X['job_description'].replace(r'>', ' ', regex=True)

In [31]:
X.sample(25)

id  \
1725  4775   
3187  7268   
1970  5609   
1185  3062   
3162  7237   
3067  7119   
2234  5948   
2560  6386   
2471  6268   
3633  7841   
514   909    
838   2106   
185   372    
3698  7909   
1090  2861   
226   462    
2921  6892   
2073  5745   
968   2276   
1749  4877   
841   2110   
2345  6087   
985   2302   
1638  4503   
103   243    

                                                                       title  \
1725  Jr. Bigdata Developer                                                    
3187  SOC Verification Engineer                                                
1970  Applied Scientist                                                        
1185  Adobe Research Graphics, Vision, Machine Learning Intern                 
3162  Lead Analytic Scientist                                                  
3067  Principal Data Scientist                                                 
2234  Tech Lead - Application Modernization                                    
2560  Sales Account Manager                                                    
2471  Default Single Point of Contact                                          
3633  Automated Driving Engineer – Advanced Simulation                         
514   Advanced Analytics Enablement Cognitive - Consultant                     
838   NERSC Exascale Science Applications Postdoctoral Fellow (NESAP)          
185   Data Scientist                                                           
3698  Sr. Solutions Architect - Artificial Intelligence and Machine Learning   
1090  Machine Learning Trading Analyst                                         
226   Data Engineer                                                            
2921  Escrow Tax Specialist II                                                 
2073  Data Warehouse Developer                                                 
968   Princ Data Science Analyst                                               
1749  Jr. Big Data Developer                                                   
841   Senior Software Engineer - Machine Learning                              
2345  Lead Data Software Development/Engineer                                  
985   Rapid Response Registered Nurse (RN)                                     
1638  Senior Analytics Consultant - Clinical Operations                        
103   Staff AI Engineer                                                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

The last regex left quite a bit of whitespace, which isn't useful. Let's strip that before moving on.

In [32]:
X['job_description'] = X['job_description'].str.strip()

In [33]:
X.sample(25)

id  \
1080  2835   
1587  4440   
418   771    
3426  7606   
2810  6719   
1892  5501   
2464  6258   
2259  5985   
634   1102   
1867  5471   
3091  7146   
2388  6134   
543   975    
1729  4797   
2400  6162   
1368  3339   
2005  5658   
252   505    
2739  6626   
2383  6129   
1001  2656   
3424  7603   
3597  7804   
793   2041   
989   2617   

                                                                    title  \
1080  Learning Analytics Program Manager                                    
1587  Data Science Intern - RISI                                            
418   Associate Vice President, Innovation Solutions                        
3426  Research Assistant II - NM                                            
2810  Legal Advocate                                                        
1892  L2 Support - Lead Engineer - Data Services                            
2464  Ice Cream Cart Attendant                                              
2259  Senior Software Engineer                                              
634   Data Scientist                                                        
1867  Associate Software Developer                                          
3091  Principal Data Scientist                                              
2388  Sr. Manager - Java Engineering Tech Lead                              
543   Open Rank Position in Bus Adm                                         
1729  Full Stack Web Developer                                              
2400  AI Speech   Language Processing Advanced Technologist                 
1368  Machine Learning Engineer                                             
2005  Marketing Data Scientist – Intelligent Customer Interactions          
252   Senior Data Scientist (NLP)                                           
2739  Corporate Trainer I                                                   
2383  Software Developer/Analyst - Big Data                                 
1001  Software Engineer, Sr Principal                                       
3424  Experimental Physicist                                                
3597  Principal Data Scientist                                              
793   Innovation Program Manager (Advisor) — AARP, Services, Inc.           
989   Applied Research Scientist - Machine Learning (PhD University Grad)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [34]:
#We don't need the ids that populated in the SQLite Database
X = X.drop(['id'], axis=1)

In [35]:
X.head(10)

title  \
0  Research Scientist - Natural Language Processing                          
1  Natural Language Processing Engineer, New York City                       
2  Research Programmer I - Natural Language Processing                       
3  Siri - Natural Language Processing Scientist/Engineer                     
4  NLP and Machine Learning Graduate                                         
5  AI Research - Natural Language Processing(NLP) Scientist                  
6  Analytical Linguist, Google AI                                            
7  IBM Research AI Residency Program                                         
8  AI Research Engineer – Natural Language Processing                        
9  Software Engineer: Machine Learning - Natural Language Processing (NLP)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [36]:
#Let's remove punctuation.
def punctuation_remover(row):
    punctuation = '!"#$&\'()*+,-./:;<=>?@[\\]^_`{|}~«»'  
    
    punctuationNoPeriod = "[" + re.sub("\.","", punctuation) + "]"
    row = re.sub(punctuationNoPeriod, " ", str(row))
    #to remove double white spaces and create space after %,
    row = row.replace('  ', ' ')
    row = row.replace('%', '% ')
    return row.strip()

<input>:5: DeprecationWarning:

invalid escape sequence \.

<input>:5: DeprecationWarning:

invalid escape sequence \.

<input>:5: DeprecationWarning:

invalid escape sequence \.

<ipython-input-36-b2830c59b60e>:5: DeprecationWarning:

invalid escape sequence \.



In [37]:
# remove punctuation from the job titles first
X['title'] = X['title'].apply(punctuation_remover)

In [38]:
X.head(10)

title  \
0  Research Scientist  Natural Language Processing                       
1  Natural Language Processing Engineer New York City                    
2  Research Programmer I  Natural Language Processing                    
3  Siri  Natural Language Processing Scientist Engineer                  
4  NLP and Machine Learning Graduate                                     
5  AI Research  Natural Language Processing NLP Scientist                
6  Analytical Linguist Google AI                                         
7  IBM Research AI Residency Program                                     
8  AI Research Engineer – Natural Language Processing                    
9  Software Engineer Machine Learning  Natural Language Processing NLP   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [39]:
#Since that worked well, let's now remove punctuation from the job descriptions.
X['job_description'] = X['job_description'].apply(punctuation_remover)

In [40]:
X.head(10)

title  \
0  Research Scientist  Natural Language Processing                       
1  Natural Language Processing Engineer New York City                    
2  Research Programmer I  Natural Language Processing                    
3  Siri  Natural Language Processing Scientist Engineer                  
4  NLP and Machine Learning Graduate                                     
5  AI Research  Natural Language Processing NLP Scientist                
6  Analytical Linguist Google AI                                         
7  IBM Research AI Residency Program                                     
8  AI Research Engineer – Natural Language Processing                    
9  Software Engineer Machine Learning  Natural Language Processing NLP   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [41]:
X['gender_breakdown'] = X['job_class']

In [42]:
X.job_class.unique()

array(['Natural Language Processing', 'Text Mining',
       'Pattern Recognition', 'Machine Learning', 'Text Analytics',
       'Apache Spark', 'Speech Recognition', 'Neural Networks'],
      dtype=object)

In [43]:
X.loc[X['gender_breakdown'] == 'Natural Language Processing', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Text Mining', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Text Analytics', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Speech Recognition', 'gender_breakdown'] = 1
X.loc[X['gender_breakdown'] == 'Pattern Recognition', 'gender_breakdown'] = 0
X.loc[X['gender_breakdown'] == 'Machine Learning', 'gender_breakdown'] = 0
X.loc[X['gender_breakdown'] == 'Apache Spark', 'gender_breakdown'] = 0
X.loc[X['gender_breakdown'] == 'Neural Networks', 'gender_breakdown'] = 0

In [44]:
X.sample(5)

title  \
786   AI Engineer Technology Development Group   
3414  Temporary Research Technician              
2346  Lead Data Scientist Solution Architect     
2210  Statistician Data Scientist                
2379  Cloud Software Engineer 5  yrs exp         

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [45]:
# to save for later
df1 = X

## Unsupervised Approach

I'll use the Latend Dirichlet Allocation method first, and then use it with bigrams and trigrams. I will evalute for coherence, and tune the models.

In [46]:
#First step, tokenizer.
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [47]:
nltk.download('wordnet')
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tiffanyfrench/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
#nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
stop_words = en_stop.union(set(['machine','business','analytics','learning','technology',
                              'system', 'science', 'research', 'analysis', 'experience',
                               'customer', 'include', 'plant', 'clerkship', 'language',
                               'university', 'google', 'hadoop', 'spark', 'apache', 'microsoft',
                               'ai', 'ml', 'nlp', 'speech', 'natural', 'engineers', 'cloud',
                               'intelligence', 'aws','ml', 'nlp', 'ai', 'azure', 'amazon']))

In [49]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [50]:
import random
text_data = []
for text in X:
    for i in range(len(X)):
        tokens = prepare_text_for_lda(X['job_description'][i])
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['believe', 'great', 'writing', 'outcome', 'focus', 'alive', 'idea', 'going', 'start', 'finish', 'document', 'three', 'minutes', 'knowing', 'respond', 'anything', 'write', 'press', 'writing', 'software', 'idea', 'center', 'certain', 'potential', 'datasets', 'everywhere', 'uncover', 'hide', 'insight', 'trend', 'story', 'people', 'strong', 'storyteller', 'textio', 'story', 'world', 'success', 'engineer', 'textio', 'skill', 'important', 'anything', 'excel', 'telling', 'compel', 'story', 'immerse', 'valuable', 'asset', 'customer', 'result', 'success', 'engineering', 'persuade', 'convince', 'build', 'relationship', 'across', 'worldwide', 'closely', 'customers’', 'textio', 'predictive', 'engine', 'pattern', 'share', 'textio', 'insight', 'company', 'usage', 'competitive', 'positioning', 'partner', 'closely', 'product', 'team', 'ensure', 'textio', 'reflect', 'happening', 'today', 'linguistic', 'landscape', 'level', 'customer', 'include', 'senior', 'executive', 'fortune', 'company', 'phenomenal

['interest', 'solving', 'interest', 'problem', 'plate', 'guild', 'epicenter', 'driving', 'utilization', 'visualization', 'statistical', 'predictive', 'solve', 'problem', 'drive', 'efficiency', 'organization', 'mission', 'delight', 'customer', 'seeking', 'talented', 'scientist', 'optimize', 'partner', 'cross', 'functional', 'team', 'develop', 'solution', 'problem', 'using', 'statistical', 'adept', 'building', 'model', 'translate', 'solution', 'something', 'matter', 'partner', 'challenge', 'possible', 'drive', 'positive', 'result', 'absorption', 'cross', 'functional', 'identify', 'solution', 'leverage', 'gather', 'requirement', 'solution', 'partner', 'translate', 'operational', 'model', 'product', 'develop', 'solution', 'utilize', 'advance', 'statistical', 'technique', 'clustering', 'regression', 'pattern', 'recognition', 'series', 'create', 'requirement', 'solution', 'include', 'source', 'training', 'frequency', 'engineering', 'develop', 'pipeline', 'promote', 'model', 'production', 'bu

['internship', 'intern', 'assist', 'functioning', 'group', 'include', 'working', 'analyst', 'integration', 'developer', 'administrator', 'develop', 'analytic', 'solution', 'alignment', 'digital', 'utility', 'solution', 'develop', 'analyze', 'system', 'include', 'maximonypa', 'transmission', 'include', 'sensorscommercial', 'operations', 'include', 'energy', 'managerfinancial', 'audit', 'relate', 'knowledge', 'skill', 'ability', 'intern', 'internship', 'knowledge', 'knowledge', 'statistical', 'analytic', 'concept', 'familiar', 'artificial', 'concept', 'knowledge', 'integration', 'volume', 'processing', 'skill', 'utilization', 'performance', 'distribute', 'system', 'nosql', 'utilization', 'analytic', 'tool', 'programming', 'python', 'utilization', 'visualization', 'tool', 'tableau', 'power', 'effectively', 'professional', 'scientist', 'financial', 'professional', 'ability', 'effectively', 'analytic', 'visualization', 'tool', 'communication', 'writing', 'skill', 'effectively', 'latest', 's

['researcher', 'involve', 'development', 'cognitive', 'immersive', 'system', 'situation', 'creation', 'relate', 'scientific', 'breakthrough', 'technical', 'innovation', 'relate', 'fields', 'learn', 'please', 'visit', 'rpijobs', 'posting', 'minimum', 'qualification', 'bachelor', 'degree', 'foreign', 'degree', 'equivalent', 'computer', 'computational', 'information', 'cognitive', 'software', 'engineering', 'development', 'relate', 'engineering', 'discipline', 'prefer', 'qualification', 'relevant', 'advance', 'degree', 'publication', 'record', 'significant', 'additional', 'years', 'professional', 'experience', 'relate', 'position', 'demonstrate', 'building', 'collaborative', 'external', 'partnershipsrelevant', 'combination', 'training', 'education', 'considere', 'minimum', 'knowledge', 'skill', 'ability', 'posse', 'passion', 'engineeringpossess', 'background', 'either', 'human', 'computer', 'interaction', 'virtual', 'reality', 'application', 'technology', 'prefer', 'knowledge', 'skill', '

['description', 'passionate', 'transformation', 'bringing', 'innovative', 'continually', 'learn', 'implement', 'technology', 'solve', 'problem', 'enjoy', 'working', 'close', 'energizing', 'environment', 'empower', 'locate', 'developer', 'analyst', 'manager', 'locate', 'triangle', 'office', 'office', 'running', 'across', 'country', 'across', 'globe', 'operate', 'agile', 'mindset', 'every', 'single', 'continually', 'improve', 'technical', 'expertise', 'delivery', 'engage', 'extract', 'transform', 'application', 'development', 'complex', 'problem', 'determination', 'number', 'source', 'emerge', 'technology', 'engineering', 'leader', 'expect', 'area', 'quickly', 'previous', 'variety', 'programming', 'language', 'protocol', 'devops', 'tool', 'demonstrate', 'agile', 'programming', 'debug', 'skill', 'exposure', 'technology', 'product', 'source', 'technology', 'evaluate', 'latest', 'technology', 'figure', 'implement', 'responsibility', 'produce', 'solution', 'ibmers', 'excel', 'performance', '

...

['description', 'overview', 'moment', 'remarkable', 'change', 'opportunity', 'convergence', 'transform', 'industry', 'society', 'workplace', 'create', 'profession', 'exist', 'emergence', 'social', 'mobile', 'global', 'services', 'leader', 'worldwide', 'transformation', 'place', 'define', 'develop', 'consult', 'career', 'whether', 'consult', 'sales', 'project', 'management', 'technical', 'opportunity', 'impact', 'world', 'working', 'solve', 'society', 'complex', 'problem', 'original', 'thinking', 'idea', 'ibmer', 'innovate', 'pursuit', 'higher', 'value', 'everything', 'guide', 'purpose', 'essential', 'essential', 'leadership', 'dedication', 'building', 'valuable', 'client', 'relationship', 'groundbreaking', 'essential', 'uncovering', 'possible', 'helping', 'global', 'client', 'succeed', 'excite', 'times', 'discover', 'moment', 'description', 'cutting', 'solution', 'bring', 'together', 'would', 'focus', 'increase', 'client', 'satisfaction', 'delivery', 'result', 'working', 'performing', 

['experience', 'scientist', 'computational', 'mathematician', 'passion', 'forefront', 'create', 'generation', 'within', 'vibrant', 'thrive', 'environment', 'lumenvox', 'would', 'looking', 'remarkable', 'individual', 'current', 'algorithm', 'signal', 'processing', 'exist', 'system', 'statistical', 'modeling', 'processing', 'information', 'retrieval', 'highly', 'desire', 'dynamic', 'position', 'require', 'exceptional', 'ability', 'technical', 'boundary', 'within', 'world', 'class', 'product', 'highly', 'skilled', 'motivate', 'individual', 'consider', 'pivotal', 'position', 'responsibility', 'program', 'primarily', 'include', 'software', 'engineering', 'algorithm', 'design', 'effectively', 'within', 'small', 'recognition', 'expert', 'mathematician', 'client', 'server', 'architect', 'identify', 'solve', 'bottleneck', 'accuracy', 'performance', 'extend', 'maintain', 'large', 'scale', 'carrier', 'grade', 'automatic', 'recognition', 'relate', 'product', 'convey', 'detail', 'documentation', 'd

['digital', 'datavoice', 'looking', 'select', 'talented', 'analyst', 'professional', 'services', 'group', 'group', 'operate', 'entrepreneurial', 'spirit', 'provide', 'class', 'consumer', 'communications', 'system', 'fortune', 'customer', 'allow', 'idea', 'strategy', 'execution', 'plan', 'hear', 'amongst', 'peer', 'practice', 'member', 'analyst', 'design', 'development', 'testing', 'product', 'analyst', 'customer', 'understand', 'document', 'requirement', 'translate', 'requirement', 'system', 'design', 'professional', 'implement', 'solution', 'deliver', 'result', 'successful', 'candidate', 'starter', 'independently', 'posse', 'critical', 'thinking', 'problem', 'solving', 'skill', 'ability', 'provide', 'recommendation', 'base', 'practice', 'excellent', 'verbal', 'write', 'communication', 'skill', 'serve', 'conduit', 'software', 'development', 'elicit', 'requirement', 'using', 'interview', 'document', 'requirement', 'workshop', 'survey', 'visit', 'process', 'description', 'case', 'scenari

['description', 'opportunity', 'everywhere', 'solve', 'problem', 'stay', 'forefront', 'artificial', 'build', 'something', 'meaningful', 'using', 'cutting', 'build', 'industry', 'leading', 'solution', 'problem', 'solver', 'solving', 'tough', 'problem', 'world', 'situation', 'thrive', 'opportunity', 'bright', 'analytical', 'engineering', 'mind', 'industry', 'share', 'passion', 'innovation', 'director', 'analytic', 'contribute', 'large', 'amount', 'world', 'nsure', 'quality', 'throughout', 'stage', 'acquisition', 'processing', 'include', 'collection', 'normalization', 'transformation', 'select', 'appropriate', 'statistical', 'method', 'computational', 'algorithm', 'build', 'oversee', 'team', 'building', 'analytic', 'model', 'relevant', 'problem', 'using', 'statistical', 'modeling', 'technique', 'manage', 'project', 'constraint', 'working', 'team', 'within', 'provide', 'quality', 'support', 'enable', 'integration', 'deployment', 'software', 'solution', 'assist', 'model', 'life', 'performin

['highly', 'specialize', 'dynamic', 'currently', 'looking', 'professional', 'tag', 'analyst', 'tag', 'analyst', 'require', 'client', 'maryland', 'dynamic', 'opportunity', 'showcase', 'skill', 'challenge', 'pace', 'environment', 'hours', 'professional', 'excellent', 'knowledge', 'problem', 'solving', 'support', 'skill', 'productively', 'collaboratively', 'others', 'effective', 'pressure', 'applicable', 'degree', 'computer', 'information', 'system', 'computer', 'database', 'administration', 'engineering', 'management', 'mathematics', 'statistics', 'another', 'quantitative', 'field', 'minimum', 'years', 'manipulate', 'building', 'statistical', 'model', 'familiarity', 'coding', 'knowledge', 'several', 'language', 'javascript', 'statistical', 'mining', 'technique', 'regression', 'random', 'forest', 'boost', 'tree', 'mining', 'social', 'network', 'query', 'database', 'using', 'statistical', 'computer', 'language', 'python', 'using', 'services', 'redshift', 'digitalocean', 'create', 'using', 

In [51]:
# Using Gensim to bolster NLTK

In [52]:
dictionary = corpora.Dictionary(text_data)

2019-03-09 10:20:58,692 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-09 10:20:58,739 : INFO : built Dictionary(4394 unique tokens: ['abstract', 'accompany', 'account', 'across', 'action']...) from 150 documents (total 37551 corpus positions)


In [53]:
dictionary.filter_extremes(no_below=10, no_above=0.2)

2019-03-09 10:20:58,755 : INFO : discarding 3916 tokens: [('abstract', 4), ('accompany', 3), ('across', 51), ('action', 38), ('alive', 1), ('always', 8), ('anything', 2), ('asset', 6), ('background', 36), ('benefit', 48)]...
2019-03-09 10:20:58,758 : INFO : keeping 478 tokens which were in no less than 10 and no more than 30 (=20.0%) documents
2019-03-09 10:20:58,763 : INFO : resulting dictionary: Dictionary(478 unique tokens: ['account', 'audience', 'become', 'believe', 'better']...)


In [54]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [55]:
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

2019-03-09 10:20:58,855 : INFO : saving Dictionary object under dictionary.gensim, separately None
2019-03-09 10:20:58,903 : INFO : saved dictionary.gensim


In [56]:
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
# Convert to array
docs = array(X['job_description'])

In [57]:
# Define function for tokenize and lemmatizing
def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [58]:
# Perform function on our document
docs = docs_preprocessor(docs)

In [59]:
# Create Biagram & Trigram Models 
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

2019-03-09 10:21:06,852 : INFO : collecting all words and their counts
2019-03-09 10:21:06,855 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-09 10:21:09,327 : INFO : collected 422880 word types from a corpus of 1335495 words (unigram + bigrams) and 3701 sentences
2019-03-09 10:21:09,328 : INFO : using 422880 counts as vocab in Phrases<0 vocab, min_count=10, threshold=10.0, max_vocab_size=40000000>
2019-03-09 10:21:09,329 : INFO : collecting all words and their counts
2019-03-09 10:21:09,333 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2019-03-09 10:21:16,765 : INFO : collected 472563 word types from a corpus of 1148603 words (unigram + bigrams) and 3701 sentences
2019-03-09 10:21:16,766 : INFO : using 472563 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [60]:
# Remove rare & common tokens 
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=10, no_above=0.2)

2019-03-09 10:21:27,769 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-09 10:21:29,506 : INFO : built Dictionary(33669 unique tokens: ['able', 'about', 'abreast', 'access', 'accommodation']...) from 3701 documents (total 1921338 corpus positions)
2019-03-09 10:21:29,599 : INFO : discarding 22851 tokens: [('able', 881), ('about', 1591), ('algorithm', 1410), ('analysis', 1719), ('analytics', 1395), ('applicant', 1216), ('application', 1817), ('apply', 1093), ('approach', 837), ('area', 1317)]...
2019-03-09 10:21:29,599 : INFO : keeping 10818 tokens which were in no less than 10 and no more than 740 (=20.0%) documents
2019-03-09 10:21:29,617 : INFO : resulting dictionary: Dictionary(10818 unique tokens: ['abreast', 'access', 'accommodation', 'accommodation_apply', 'accommodation_request']...)


In [61]:
#Create dictionary and corpus required for Topic Modeling
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))
print(corpus[:1])

Number of unique tokens: 10818
Number of documents: 3701
[[(0, 1), (1, 1), (2, 3), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 6), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 3), (25, 1), (26, 3), (27, 1), (28, 1), (29, 2), (30, 3), (31, 1), (32, 1), (33, 1), (34, 3), (35, 1), (36, 1), (37, 3), (38, 2), (39, 1), (40, 1), (41, 4), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 3), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 3), (69, 1), (70, 2), (71, 3), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 2), (89, 4), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (

In [62]:
# Set parameters.
num_topics = 5
chunksize = 500 
passes = 20 
iterations = 400
eval_every = 1  

In [63]:
# Make a index to word dictionary.
temp = dictionary[0]  # only to "load" the dictionary.
id2word = dictionary.id2token

lda_model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

2019-03-09 10:21:30,795 : INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
2019-03-09 10:21:30,798 : INFO : using serial LDA version on this node
2019-03-09 10:21:30,806 : INFO : running online (multi-pass) LDA training, 5 topics, 20 passes over the supplied corpus of 3701 documents, updating model once every 500 documents, evaluating perplexity every 500 documents, iterating 400x with a convergence threshold of 0.001000
2019-03-09 10:21:34,213 : INFO : -9.822 per-word bound, 905.2 perplexity estimate based on a held-out corpus of 500 documents with 125353 words
2019-03-09 10:21:34,213 : INFO : PROGRESS: pass 0, at document #500/3701
2019-03-09 10:21:37,263 : INFO : optimized alpha [0.12530613, 0.06261243, 0.07075375, 0.066852495, 0.093186066]
2019-03-09 10:21:37,265 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-09 10:21:37,271 : INFO : topic #0 (0.125): 0.007*"data_scientist" + 0.003*"artificial_intelligence" + 0.003*"collabora

2019-03-09 10:21:45,566 : INFO : -10.183 per-word bound, 1162.7 perplexity estimate based on a held-out corpus of 500 documents with 154558 words
2019-03-09 10:21:45,566 : INFO : PROGRESS: pass 0, at document #2500/3701
2019-03-09 10:21:46,265 : INFO : optimized alpha [0.24287005, 0.17436466, 0.16179955, 0.11934261, 0.27512068]
2019-03-09 10:21:46,266 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-09 10:21:46,272 : INFO : topic #0 (0.243): 0.008*"data_scientist" + 0.004*"collaborate_with" + 0.003*"familiarity_with" + 0.003*"large_scale" + 0.003*"speech_recognition" + 0.003*"data_set" + 0.002*"software_engineer" + 0.002*"programming_language" + 0.002*"artificial_intelligence" + 0.002*"around_world"
2019-03-09 10:21:46,274 : INFO : topic #1 (0.174): 0.024*"apache_spark" + 0.008*"apache" + 0.004*"data_warehouse" + 0.004*"data_set" + 0.004*"large_scale" + 0.003*"software_development" + 0.003*"cross_functional" + 0.003*"hadoop" + 0.003*"amazon" + 0.003*"e

2019-03-09 10:21:52,141 : INFO : topic #1 (0.201): 0.005*"apache_spark" + 0.005*"data_set" + 0.004*"large_scale" + 0.004*"cross_functional" + 0.003*"software_development" + 0.003*"data_mining" + 0.003*"this_role" + 0.003*"amazon" + 0.003*"communication_skill" + 0.003*"data_scientist"
2019-03-09 10:21:52,143 : INFO : topic #2 (0.142): 0.004*"essential_function" + 0.003*"others" + 0.003*"equipment" + 0.003*"clinical" + 0.003*"duty" + 0.003*"care" + 0.003*"staff" + 0.002*"procedure" + 0.002*"policy_procedure" + 0.002*"essential"
2019-03-09 10:21:52,145 : INFO : topic #3 (0.131): 0.006*"without_regard" + 0.006*"other_characteristic" + 0.005*"qualified_applicant" + 0.005*"will_receive" + 0.005*"physical_mental" + 0.005*"consideration_employment" + 0.004*"microsoft" + 0.004*"gender_identity" + 0.004*"accommodation" + 0.003*"race_color"
2019-03-09 10:21:52,147 : INFO : topic #4 (0.339): 0.004*"data_scientist" + 0.004*"software_development" + 0.004*"artificial_intelligence" + 0.004*"this_role"

2019-03-09 10:21:58,045 : INFO : topic #3 (0.137): 0.007*"without_regard" + 0.007*"qualified_applicant" + 0.006*"other_characteristic" + 0.006*"will_receive" + 0.006*"consideration_employment" + 0.005*"physical_mental" + 0.005*"microsoft" + 0.005*"gender_identity" + 0.005*"reasonable_accommodation" + 0.005*"accommodation"
2019-03-09 10:21:58,047 : INFO : topic #4 (0.538): 0.005*"software_development" + 0.004*"open_source" + 0.004*"software_engineer" + 0.004*"best_practice" + 0.004*"this_role" + 0.003*"data_scientist" + 0.003*"least_year" + 0.003*"team_member" + 0.003*"apache" + 0.003*"real_time"
2019-03-09 10:21:58,049 : INFO : topic diff=0.286676, rho=0.326129
2019-03-09 10:21:59,079 : INFO : -8.330 per-word bound, 321.7 perplexity estimate based on a held-out corpus of 500 documents with 214331 words
2019-03-09 10:21:59,079 : INFO : PROGRESS: pass 1, at document #3000/3701
2019-03-09 10:21:59,570 : INFO : optimized alpha [0.33244827, 0.2661452, 0.17453904, 0.14410795, 0.3816166]
2019

2019-03-09 10:22:03,096 : INFO : topic diff=0.237501, rho=0.310057
2019-03-09 10:22:04,032 : INFO : -8.372 per-word bound, 331.2 perplexity estimate based on a held-out corpus of 500 documents with 155674 words
2019-03-09 10:22:04,033 : INFO : PROGRESS: pass 2, at document #1000/3701
2019-03-09 10:22:04,541 : INFO : optimized alpha [0.48800886, 0.2722141, 0.14530282, 0.14195773, 0.46449888]
2019-03-09 10:22:04,541 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-09 10:22:04,548 : INFO : topic #0 (0.488): 0.009*"data_scientist" + 0.007*"neural_network" + 0.005*"computer_vision" + 0.004*"artificial_intelligence" + 0.004*"familiarity_with" + 0.004*"data_mining" + 0.004*"pattern_recognition" + 0.003*"large_scale" + 0.003*"preferred_qualification" + 0.003*"collaborate_with"
2019-03-09 10:22:04,549 : INFO : topic #1 (0.272): 0.007*"data_set" + 0.005*"data_mining" + 0.004*"problem_solving" + 0.004*"cross_functional" + 0.004*"advanced_analytics" + 0.003*"minin

...

2019-03-09 10:24:11,600 : INFO : topic #0 (0.417): 0.006*"artificial_intelligence" + 0.006*"computer_vision" + 0.005*"familiarity_with" + 0.005*"neural_network" + 0.005*"speech_recognition" + 0.003*"full_time" + 0.003*"preferred_qualification" + 0.003*"large_scale" + 0.003*"signal_processing" + 0.003*"electrical_engineering"
2019-03-09 10:24:11,602 : INFO : topic #1 (0.390): 0.010*"apache_spark" + 0.009*"data_scientist" + 0.006*"data_set" + 0.005*"data_mining" + 0.004*"advanced_analytics" + 0.004*"problem_solving" + 0.004*"bachelor_degree" + 0.004*"cross_functional" + 0.003*"communication_skill" + 0.003*"risk"
2019-03-09 10:24:11,603 : INFO : topic #2 (0.134): 0.005*"others" + 0.005*"essential_function" + 0.004*"equipment" + 0.003*"duty" + 0.003*"rule" + 0.003*"high_school" + 0.003*"policy_procedure" + 0.003*"essential" + 0.003*"must_able" + 0.003*"procedure"
2019-03-09 10:24:11,605 : INFO : topic #3 (0.155): 0.011*"without_regard" + 0.010*"gender_identity" + 0.008*"race_color" + 0.008

2019-03-09 10:24:15,061 : INFO : topic #2 (0.127): 0.005*"essential_function" + 0.005*"others" + 0.004*"policy_procedure" + 0.004*"equipment" + 0.004*"duty" + 0.004*"procedure" + 0.003*"essential" + 0.003*"high_school" + 0.003*"care" + 0.003*"appropriate"
2019-03-09 10:24:15,063 : INFO : topic #3 (0.159): 0.011*"without_regard" + 0.010*"gender_identity" + 0.009*"race_color" + 0.009*"qualified_applicant" + 0.008*"other_characteristic" + 0.007*"will_receive" + 0.007*"consideration_employment" + 0.007*"equal_employment" + 0.006*"marital_status" + 0.006*"receive"
2019-03-09 10:24:15,065 : INFO : topic #4 (0.523): 0.006*"software_development" + 0.005*"software_engineer" + 0.004*"this_role" + 0.004*"large_scale" + 0.003*"least_year" + 0.003*"open_source" + 0.003*"passionate_about" + 0.003*"real_time" + 0.003*"more_than" + 0.003*"cutting_edge"
2019-03-09 10:24:15,066 : INFO : topic diff=0.098164, rho=0.194617
2019-03-09 10:24:15,690 : INFO : -8.292 per-word bound, 313.4 perplexity estimate ba

2019-03-09 10:24:20,017 : INFO : topic #4 (0.494): 0.007*"software_development" + 0.005*"software_engineer" + 0.005*"apache" + 0.004*"open_source" + 0.004*"least_year" + 0.004*"this_role" + 0.004*"real_time" + 0.004*"large_scale" + 0.003*"passionate_about" + 0.003*"team_member"
2019-03-09 10:24:20,019 : INFO : topic diff=0.189892, rho=0.194617
2019-03-09 10:24:20,706 : INFO : -8.268 per-word bound, 308.3 perplexity estimate based on a held-out corpus of 500 documents with 144175 words
2019-03-09 10:24:20,707 : INFO : PROGRESS: pass 18, at document #3500/3701
2019-03-09 10:24:20,993 : INFO : optimized alpha [0.45608932, 0.39595568, 0.1348959, 0.15872245, 0.49699333]
2019-03-09 10:24:20,994 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-09 10:24:21,001 : INFO : topic #0 (0.456): 0.009*"neural_network" + 0.007*"computer_vision" + 0.006*"artificial_intelligence" + 0.005*"familiarity_with" + 0.004*"speech_recognition" + 0.003*"full_time" + 0.003*"preferre

2019-03-09 10:24:24,110 : INFO : PROGRESS: pass 19, at document #1500/3701
2019-03-09 10:24:24,394 : INFO : optimized alpha [0.56814814, 0.43774495, 0.12512372, 0.16065553, 0.5598957]
2019-03-09 10:24:24,395 : INFO : merging changes from 500 documents into a model of 3701 documents
2019-03-09 10:24:24,402 : INFO : topic #0 (0.568): 0.007*"computer_vision" + 0.006*"neural_network" + 0.006*"artificial_intelligence" + 0.005*"familiarity_with" + 0.004*"pattern_recognition" + 0.003*"full_time" + 0.003*"preferred_qualification" + 0.003*"real_world" + 0.003*"internship" + 0.003*"large_scale"
2019-03-09 10:24:24,403 : INFO : topic #1 (0.438): 0.011*"data_scientist" + 0.008*"data_set" + 0.007*"data_mining" + 0.005*"advanced_analytics" + 0.004*"problem_solving" + 0.004*"predictive" + 0.004*"mining" + 0.004*"cross_functional" + 0.003*"bachelor_degree" + 0.003*"apache_spark"
2019-03-09 10:24:24,407 : INFO : topic #2 (0.125): 0.005*"essential_function" + 0.005*"others" + 0.004*"equipment" + 0.004*"

2019-03-09 10:24:28,575 : INFO : topic #1 (0.399): 0.010*"data_scientist" + 0.008*"apache_spark" + 0.007*"data_set" + 0.006*"data_mining" + 0.004*"advanced_analytics" + 0.004*"problem_solving" + 0.004*"bachelor_degree" + 0.004*"predictive" + 0.004*"cross_functional" + 0.003*"marketing"
2019-03-09 10:24:28,577 : INFO : topic #2 (0.135): 0.005*"essential_function" + 0.005*"others" + 0.004*"equipment" + 0.004*"policy_procedure" + 0.003*"duty" + 0.003*"essential" + 0.003*"procedure" + 0.003*"care" + 0.003*"high_school" + 0.003*"rule"
2019-03-09 10:24:28,578 : INFO : topic #3 (0.159): 0.010*"without_regard" + 0.010*"gender_identity" + 0.008*"race_color" + 0.008*"other_characteristic" + 0.008*"qualified_applicant" + 0.007*"will_receive" + 0.006*"equal_employment" + 0.006*"consideration_employment" + 0.006*"marital_status" + 0.006*"microsoft"
2019-03-09 10:24:28,580 : INFO : topic #4 (0.500): 0.007*"software_development" + 0.005*"software_engineer" + 0.004*"apache" + 0.004*"this_role" + 0.004

In [64]:
# Print the Keyword in the 5 topics
#print(lda_model.print_topics())

In [65]:
id2word = gensim.corpora.Dictionary(text_data)

2019-03-09 10:24:29,031 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-09 10:24:29,113 : INFO : built Dictionary(4394 unique tokens: ['abstract', 'accompany', 'account', 'across', 'action']...) from 150 documents (total 37551 corpus positions)


In [66]:
#id2word.filter_extremes(no_below=20, no_above=.1)

In [67]:
#bow_collection = list(map(id2word.doc2bow, text_data))

In [68]:
#print(bow_collection)

In [69]:
lda = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=100,
)

2019-03-09 10:24:29,151 : INFO : using symmetric alpha at 0.01
2019-03-09 10:24:29,152 : INFO : using symmetric eta at 0.01
2019-03-09 10:24:29,156 : INFO : using serial LDA version on this node
2019-03-09 10:24:29,282 : INFO : running online (single-pass) LDA training, 100 topics, 1 passes over the supplied corpus of 3701 documents, updating model once every 2000 documents, evaluating perplexity every 3701 documents, iterating 50x with a convergence threshold of 0.001000
2019-03-09 10:24:29,283 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-03-09 10:24:29,285 : INFO : PROGRESS: pass 0, at document #2000/3701
2019-03-09 10:24:33,089 : INFO : merging changes from 2000 documents into a model of 3701 documents
2019-03-09 10:24:33,214 : INFO : topic #63 (0.010): 0.005*"security" + 0.003*"more_than" + 0.003*"data_scientist" + 0.003*"data_mining" + 0.003*"problem_solving" + 0.003*"minimum_year" + 0.003

In [70]:
#This is a job description for a deep-learning engineer. This field has about 66% men, and is among the most balanced.
one = '''Job Summary:
Reporting to the AI Science Leader the responsibility of this position is to create products or tools built on Artificial Intelligence (specifically natural language processing, and information extraction) technologies and paradigms, and which can enable high-value offerings in one or more of our Service Lines.

Essential Functions of the Job:
In collaboration with internal domain experts and his/her network of AI technology experts, develop concepts for AI-based products
Define, design and develop client-focused AI or cognitive solutions leveraging Natural Language Processing (NLP), Machine Learning, Probabilistic Decision Making and related methods and paradigms, and supporting technology
Serve as a Senior AI scientist and Subject Matter Professional (SMP), in the face of internal and external stakeholders, to help establish EY’s AI competency
Work closely with various industry and functional teams to incorporate AI-based capabilities into solution offerings or enablement assets drive value for our clients, across all of our service lines
Perform AI research and development (R&D) work, develop prototypes of AI solutions and assets to support AI solution development tasks
Supervision Responsibilities:
Works independently and requires limited supervision; excels as an individual contributor as part of a team of scientists and software engineers
Knowledge and Skills Requirements:
Expert-level background in Deep Learning with understanding of theories and applications of Deep Learning and Deep Learning literature
Hands-on experience or contributed to Keras, TensorFlow, PyTorch or other Deep Learning toolkit and libraries
Drive to solve customers’ problems using technical creativity and hard work
Ability to communicate technical matters clearly and effectively to both technical and non-technical audiences, win supporters and spur people to action
Excellent consultative, listening, communications and analytical skills
Excellent interpersonal skills, including the ability to work across the organization and interact/influence/negotiate effectively at all levels of management and peers
Critical thinker with excellent judgment and strong initiative
Recognized expertise in an artificial intelligence and machine learning applications as evidenced by relevant publications or awards, preferred
Education:
Graduate degree (PhD preferred) in computer science, computer engineering, electrical engineering or related fields (statistics, applied math, computational neuroscience)
Experience:
Minimum of 5 years in the market, with relevant Deep Learning, AI, ML design and build experience
Experience in the lifecycle of enterprise-class software development
EY, an equal employment opportunity employer (Females/Minorities/Protected Veterans/Disabled), values the diversity of our workforce and the knowledge of our people.'''

In [71]:
?lda.get_document_topics

In [72]:
bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(one))
topics = lda.get_document_topics(bow_lda, minimum_probability=0)

In [73]:
for key, contribution in topics:
    print(f'\n:: {key} ({contribution*100:.2}%)')
    for i, __ in lda.get_topic_terms(key, topn=5):
      if i in id2word:
        print(f' {id2word[i]}', end=',')


:: 0 (0.0057%)
 automatic, orchestration,
:: 1 (8.4%)
 recruit, ability, trainer, guideline,
:: 2 (0.0057%)
 boost, recruit, powerpoint, afternoon, automatic,
:: 3 (0.0057%)
 scotty, automatic, novel, office, affordably,
:: 4 (0.0057%)
 linuxproficiency, amaze, bridges, details,
:: 5 (0.0057%)
 productively, northropgrumman,
:: 6 (0.0057%)
 boat, describe,
:: 7 (0.0057%)
 optimal, researcher, novel, airport,
:: 8 (0.0057%)
 mining, optimize, scotty,
:: 9 (0.0057%)
 contribute, dining, jefferson, guild, cyber,
:: 10 (0.0057%)
 wearing, zillow,
:: 11 (0.0057%)
 malfunction, expansion, nexus, thoughtful, completion,
:: 12 (0.0057%)
 designing, interface, researcher,
:: 13 (0.0057%)
 researcher, inquiry, malware, contract,
:: 14 (0.0057%)
 trainee,
:: 15 (0.0057%)
 thoughtful, last,
:: 16 (3.8%)
 automatic, exercising,
:: 17 (4.9%)
 logistics, electricity, electrical,
:: 18 (0.0057%)
 take, pride, nexus,
:: 19 (0.0057%)
 contract, society, powerpoint, record,
:: 20 (0.0057%)
 contract, vi

In [74]:
??lda.get_document_topics

In [75]:
??lda.inference

In [77]:
bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(one))
gamma, phi = lda.inference([bow_lda])
one_topics = gamma[0] / gamma[0].sum()
one_topics

array([5.7142857e-05, 8.3472103e-02, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       3.7719946e-02, 5.0446525e-02, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 1.6695913e-02, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 9.4440602e-02, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 7.9738401e-02,
       5.7142857e-05, 1.2303044e-01, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 5.7142857e-05,
       5.7142857e-05, 5.7142857e-05, 5.7142857e-05, 1.1396844e

In [78]:
#NLP Position: https://www.indeed.com/viewjob?jk=354e5b916d61947d&tk=1d5c32trh404o803&from=serp&vjs=3
two = """Are you excited at the prospect of unlocking the secrets held by a data set? Are you fascinated by the possibilities presented by the IoT, machine learning, and artificial intelligence advances? In an increasingly connected world, massive amounts of structured and unstructured data open up new opportunities. As a data scientist, you can turn these complex data sets into useful information to solve global challenges. Across private and public sectors — from fraud detection, to cancer research, to national intelligence — you know the answers are in the data. 

We have an opportunity for you to use your leadership and analytical skills to improve the private and public clients we support. You’ll work closely with your customer to understand their questions and needs, and then dig into their data-rich environment to find the pieces of their information puzzle. You’ll mentor teammates, develop algorithms, write scripts, build predictive analytics, use automation, apply machine learning, and use the right combination of tools and frameworks to turn that set of disparate data points into objective answers to help senior leadership make informed decisions. You’ll provide your customer with a deep understanding of their data, what it all means, and how they can use it. Join us as we use data science for good in both the private and public sectors.

Empower change with us. 

Build Your Career:
At Booz Allen, we know the power of analytics and we’re dedicated to helping you grow as a data analysis professional. When you join Booz Allen, you can expect: 
access to online and onsite training in data analysis and presentation methodologies, and tools like Hortonworks, Docker, Tableau, and Splunk 
a chance to change the world with the Data Science Bowl—the world’s premier data science for social good competition 
participation in partnerships with data science leaders, like our partnership with NVIDIA to deliver Deep Learning Institute (DLI) training to the federal government 

You’ll have access to a wealth of training resources through our Analytics University, an online learning portal specifically geared towards data science and analytics skills, where you can access more than 5000 functional and technical courses, certifications, and books. Build your technical skills through hands-on training on the latest tools and state-of-the-art tech from our in-house experts. Pursuing certifications? Take advantage of our tuition assistance, on-site bootcamps, certification training, academic programs, vendor relationships, and a network of professionals who can give you helpful tips. We’ll help you develop the career you want, as you chart your own course for success. 

You Have:
4+ years of experience with developing statistical and machine learning models using R or Python
Experience with developing algorithms to analyze text data, including natural language processing NLP
Experience with design, development, testing, and implementation of new functionality for reporting
Ability to write scripts and compile and analyze data from relational databases specific to customer experience and surveys
Ability to obtain a security clearance
BA or BS degree
Nice If You Have:
PhD in CS, Cognitive Science, Computational Linguistics, Informatics, or related field preferred
Clearance:
Applicants selected will be subject to a security investigation and may need to meet eligibility requirements for access to classified information. 

We’re an EOE that empowers our people—no matter their race, color, religion, sex, gender identity, sexual orientation, national origin, disability, veteran status, or other protected characteristic—to fearlessly drive change."""

bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(two))
gamma, phi = lda.inference([bow_lda])
two_topics = gamma[0] / gamma[0].sum()
two_topics

array([4.5871562e-05, 2.6067775e-02, 9.9023744e-02, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 7.1751155e-02, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 9.8706469e-02,
       4.5871562e-05, 1.2258393e-01, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e-05,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 3.2174252e-02,
       4.5871562e-05, 4.5871562e-05, 4.5871562e-05, 4.5871562e

In [79]:
# Pattern Recognition: https://www.indeed.com/viewjob?jk=974d4b7f90a8ecf3&q=pattern+recognition&tk=1d5c3hkl9av4g803&from=web&vjs=3
three = """
Concept Tapestry LLC is seeking image processing and pattern recognition programmers to join our team. The successful candidate must thrive on developing and implementing outstanding software. The pace of work is fast and is focused on generating new ideas and developing proof of concept systems that that showcase advancements in machine learning. We are looking for image processing software engineers to advance our development (Full or Part-Time.) We offer a complete benefits package and a very flexible and rewarding work environment.
This is a hands on software development position requiring skilled Java programming and strong knowledge of computer science technologies.
Responsibilities

Develop and implement algorithms and code for analyzing large image data sets using image analysis algorithms.
Develop proof of concept applications to showcase and test out new ideas and algorithms for feature extraction and image analysis.
Build flexible platform for performing image analysis on large data sets
Perform analysis on customer data using image processing platform to solve image classification and image analysis problems
Work with team to improve existing algorithms for scaling up performance – faster processing and handling larger volume of images.
Desired experience and skills

Degree in Computer Science/Mathematics.
Experience and interest in using pattern recognition and image processing techniques to analyze large sets of images
Experience in developing programs for analyzing images over noisy and uncertain image data sets
Experience working on developing AI based human vision technologies.
Skilled Java programming.
Excellent mathematics/statistics background with hands on experience using image analysis tools and libraries.
Experience building data driven image classification models.
Experience creating data visualization tools.
Experience in developing real-time image analysis systems.
"""

bow_lda = id2word.doc2bow(gensim.utils.simple_preprocess(three))
gamma, phi = lda.inference([bow_lda])
three_topics = gamma[0] / gamma[0].sum()
three_topics

array([7.3529423e-05, 7.3529423e-05, 2.0008014e-01, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 1.3243818e-01, 7.3529423e-05, 7.3529423e-05,
       2.4310466e-02, 7.3529423e-05, 7.3529423e-05, 2.1365890e-02,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 2.1968698e-02, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 2.4266854e-01,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 7.3529423e-05, 7.3529423e-05,
       7.3529423e-05, 7.3529423e-05, 2.6541356e-02, 7.3529423e

### The distance between the Deep Learning Engineer (66% male) and the NLP Position (mostly women).

In [82]:
scipy.spatial.distance.pdist([one_topics, two_topics])

array([0.25583754])

### The distance between the Deep Learning and Pattern Recogition (98% male).

In [83]:
scipy.spatial.distance.pdist([one_topics, three_topics])

array([0.41192855])

### The distance between NLP and Pattern Recognition.

In [84]:
scipy.spatial.distance.pdist([two_topics, three_topics])

array([0.3271281])

### This is very interesting.  What we can see here is that a job description from a relatively balanced field has the greatest distance from the very-male dominated field.  While the NLP and Pattern Recognition aren't as far apart as the Deep Learning and Pattern Recognition, I think it is still of note.  What we see here is that a job description from a balanced field is different from one that is not balanced at all.  This makes a great case that even job descriptions that might be skewed toward one gender or another are more similar than we realize as well.  
### In the end, I think this makes a great case for gender-balanced job descriptions that can attract the best and brightest from any gender (I'd, of course, like to broaden this to people who identify as gender non-binary).
### I think something like this projection could help create a program that analyzes a job description, and gives feedback to the client about how balanced it might be.  It would take web development, and extensive model training, but I think something like that could be valuable.
### This projection model was created with extensive help from my mentor, Philip Robinson.  So I'd like to give him credit here where it's due!